In [70]:
import time
import hyperloglog
import numpy as np
import random

## 1. HASHING

In [71]:
real_len = 125000001 # obtained with set()

### Built-in HLL

In [72]:
def HLL(file,nlines):
    f = open(file, "r")
    hll = hyperloglog.HyperLogLog(0.01)  # accept 1% counting error
    for n in range(1,nlines):
        req = f.readline(n)
        hll.add(req)
    
    return len(hll) 

In [73]:
start = time.time()
card = HLL('hash.txt',139000000)
end = time.time()

In [74]:
print ('Execution time:',end-start, ' seconds')
print('Error obatined with the built-in hyperloglog is:',((real_len-card)/card)*100)

Execution time: 216.43159770965576  seconds
Error obatined with the built-in hyperloglog is: 0.06426126807226587


### My_HLL
1. For each request we compute the hash. We generate a random hash function that maps strings to a 32 bits vector. It will gives us a uniform distribution across the number of requests. 

3. The first m bits in this vector represents the bucket to which we should assign a certain request. If we have for example p = 10 we should consider the first 1024 bits since $2^{10} = 1024$, which is the total number of combination of the values [0,1] for 10 digits. Therefore the number of buckets will always be a power of 2.

4. If we have a good and random hash function that acted on strings and generated binary vectors we would expect: $1/2^n$ of them to have their binary representation starting in $0*n$. Therefore we determine the number of leftmost zeros for each request. If the number of leftmost zeros for the $i*{th}$ element is *k* and it is higher than the max *k* generated so far, *k* is stored as the new max value for that bucket.

5. Once input is exhausted, we call the cardinality function on all the buckets. We calculate the number of unique words with the following formula:

    ${\displaystyle E=\alpha _{m}m^{2}Z}$ \
     \
    Where: 
    
    $Z=\sum _{j=1}^{m}{2^{-M[j]}}^{-1}$ and $\alpha _{m} = 0.7213/(1+1.079/(2^{m}))$ 
    
 
6. The observed error depends on the number of buckets m and it is given by the formula: \
    $ error=\frac{1.04}{\sqrt(2^p)}$ \
    where *p* is the exponent we give to 2 to obtain *m*.


In [75]:
def hash_fun(string,n):
    m = 2**n
    if len(string)!=0:
        random.seed(20) 
        x,y,z=random.sample(range(1,m),3)
        hashed_up=(int(string[:13],16)*x + int(string[13:26],16)*y + int(string[26:],16)*z) % m
        l=len(bin(hashed_up)[2:])
        return '0'*(n-l)+bin(hashed_up)[2:]
    else:
        return'0'*n

def my_hll(file,p,n_lines):     
    f = open(file, "r")
    m = 2**p
    buckets = [0]*m
    for request in range(35,n_lines):
        line = f.readline(request)
        binary = hash_fun(line,32)
        index = int(binary[:p],2) 
        end = binary[p:] 
        buckets[index]=max(buckets[index],leftmost_zeros(end))

    card = cardinality(buckets,m)

    return (card)

def cardinality(buckets,m):
    Z = 1/sum([2**(-bucket-1) for bucket in buckets])
    alfa_m = 0.7213/(1+1.079/(m))
    E = alfa_m*(m**2)*Z
    return E

def leftmost_zeros(b):
    i = 0
    while b[i] == '0' :
        i+=1
        if len(b[i:]) == 0:
            break
    return i

In [76]:
start = time.time()
my_card = my_hll('hash.txt',12,139000000)
end = time.time()

In [8]:
print(' Execution time is:',round((end-start)/60,2), ' minutes' )
print(' The estimated cardinality is:', round(my_card,0))
print(' Experimental error:',round((real_len-my_card)/(real_len)*100,3), '%')

 Execution time is: 23.18  minutes
 The estimated cardinality is: 124212668
 Experimental error: 0.63 %


---------------

### 3.Algorithmic question
You are given an array with A with n integer numbers. <br>
Let s = min{ A[1], ..., A[n] } and b = max { A[1], ..., A[n] }. <br>
Let r = b - s <br>
Prove that we can sort A in time O(n + r). <br>
<br>
We know that we have integers between b and s and the difference between them is r. So we can consider Bucket sort or bin sort which is a sorting algorithm that works by distributing the elements of an array into a number of buckets. Each bucket is then sorted individually, either using a different sorting algorithm, or by recursively applying the bucket sorting algorithm. But since here we have integers we should not worry about 2 or more different objects being put in the same bucket. This is how it works: <br>
At first we have to create r+1 buckets after we have to scroll down our array for each element and assign each integer to its specific bucket so the runtime for this operation is just n, Then we have to join and combine all the buckets together so one more r+1. The total run time for this algorithm is:$O(2(r+1)+n)=O(r+n)$ <br>
Below we also see how it works as well:
First we create Buckets: $\;\;\;\;\;\;$    $Bucket[s],...,Bucket[b]$ <br>
Then we scroll down the list from $A[1]$ to $A[n]$ and assign each ineger to a bucket and since we have max and min all the other integers will be surely included. <br>
And then we simply combine the buckets in order of $Bucket[s]$ to $ Bucket[b]$. And we will have our final array:<br> 
$Sorted\;array=[elements \; in \; Bucket[s],...,elements \; in \; Bucket[b]]$